In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split


In [2]:
path = 'California_Fire_Incidents.xlsx'
fire = pd.read_excel(path)
columns = ['AcresBurned', 'AirTankers', 'AdminUnit', 'Counties', 'CountyIds', 'Engines', 'Latitude', 'Longitude', 'Started']

In [3]:
fire_df = fire[columns]
fire_df

,AcresBurned,AirTankers,AdminUnit,Counties,CountyIds,Engines,Latitude,Longitude,Started
0,257314.0,NaN,Stanislaus National Forest/Yosemite National Park,Tuolumne,55,NaN,37.857000,-120.086000,2013-08-17T15:25:00Z
1,30274.0,NaN,USFS Angeles National Forest/Los Angeles Count...,Los Angeles,19,NaN,34.585595,-118.423176,2013-05-30T15:28:00Z
2,27531.0,NaN,CAL FIRE Riverside Unit / San Bernardino Natio...,Riverside,33,NaN,33.709500,-116.728850,2013-07-15T13:43:00Z
3,27440.0,NaN,Tahoe National Forest,Placer,31,NaN,39.120000,-120.650000,2013-08-10T16:30:00Z
4,24251.0,NaN,Ventura County Fire/CAL FIRE,Ventura,56,117.0,0.000000,0.000000,2013-05-02T07:01:00Z
...,...,...,...,...,...,...,...,...,...
1631,9.0,NaN,CAL FIRE / Riverside County Fire,Riverside,33,NaN,33.827979,-117.499619,2019-10-10T12:08:00Z
1632,2.0,NaN,CAL FIRE Nevada-Yuba-Placer Unit,Nevada,29,NaN,39.409722,-121.000556,2019-06-28T15:03:04Z
1633,NaN,NaN,Yolo County Fire Protection District,Yolo,57,NaN,38.734634,-121.729691,2019-11-25T12:02:02Z
1634,NaN,NaN,Camp Pendleton Marine Corps Base,San Diego,37,NaN,33.351145,-117.403719,2019-10-22T19:20:44Z


In [4]:
fire_df.isna().sum()

AcresBurned       3
AirTankers     1608
AdminUnit         0
Counties          0
CountyIds         0
Engines        1445
Latitude          0
Longitude         0
Started           0
dtype: int64

In [5]:
fire_df = fire_df.drop(['AirTankers', 'Engines', 'AdminUnit'], axis=1)
fire_df = fire_df.dropna()

In [6]:
fire_df.head()

,AcresBurned,Counties,CountyIds,Latitude,Longitude,Started
0,257314.0,Tuolumne,55,37.857000,-120.086000,2013-08-17T15:25:00Z
1,30274.0,Los Angeles,19,34.585595,-118.423176,2013-05-30T15:28:00Z
2,27531.0,Riverside,33,33.709500,-116.728850,2013-07-15T13:43:00Z
3,27440.0,Placer,31,39.120000,-120.650000,2013-08-10T16:30:00Z
4,24251.0,Ventura,56,0.000000,0.000000,2013-05-02T07:01:00Z


In [7]:
fire_df.dtypes

AcresBurned    float64
Counties        object
CountyIds       object
Latitude       float64
Longitude      float64
Started         object
dtype: object

In [8]:
months = []
years = []
for index, row in fire_df.iterrows():
   #print(date)
    month = row['Started'].split('-')[1]
    year = row['Started'].split('-')[0]
    months.append(np.float32(month))
    years.append(np.float32(year))

In [9]:
fire_df = fire_df.drop('Started', axis=1)
fire_df['Month'] = months
fire_df['Year'] = years
fire_df.head()

,AcresBurned,Counties,CountyIds,Latitude,Longitude,Month,Year
0,257314.0,Tuolumne,55,37.857000,-120.086000,8.0,2013.0
1,30274.0,Los Angeles,19,34.585595,-118.423176,5.0,2013.0
2,27531.0,Riverside,33,33.709500,-116.728850,7.0,2013.0
3,27440.0,Placer,31,39.120000,-120.650000,8.0,2013.0
4,24251.0,Ventura,56,0.000000,0.000000,5.0,2013.0


In [10]:
labels = fire_df['Counties']
features = fire_df.drop(['Counties', 'CountyIds'], axis=1)

In [11]:
one_hot_labels = pd.get_dummies(labels).to_numpy()
scaler = StandardScaler()
acres = features['AcresBurned'].to_numpy().reshape(-1,1)
features['AcresBurned'] = scaler.fit_transform(acres)
features.head()

,AcresBurned,Latitude,Longitude,Month,Year
0,9.271578,37.857000,-120.086000,8.0,2013.0
1,0.942276,34.585595,-118.423176,5.0,2013.0
2,0.841645,33.709500,-116.728850,7.0,2013.0
3,0.838307,39.120000,-120.650000,8.0,2013.0
4,0.721314,0.000000,0.000000,5.0,2013.0


In [12]:
features.to_csv('cleaned_firedata.csv')

In [13]:
print(features.shape)
one_hot_labels.shape

(1633, 5)


(1633, 59)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(features, one_hot_labels, test_size=0.2)

## Rough draft model


In [15]:
import sys
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
# from tensorflow import keras
from tensorflow.keras import layers, Sequential, Input
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, concatenate, BatchNormalization
# from tensorflow.keras.layers.core import *
from tensorflow.keras.models import Model
from tensorflow.keras.layers.experimental import preprocessing
# from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input


def build_dnn():

    
    normalizer = layers.BatchNormalization(momentum=0.98, epsilon=.001, trainable=True)

    model = Sequential([

        #normalizer,
        layers.Dense(24, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(1)
    ])
    model = Model([img_input, num_input], out)
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0001), metrics=['accuracy'])
    print("Model compiled")
    return model

In [42]:
def build_dnn(input_shape, numclasses):
    feature_input = Input(shape=input_shape)
    model = Dense(64, activation='relu')(feature_input)
    model = Dense(16, activation='relu')(model)
    model = Dense(4, activation='relu')(model)
    output = Dense(numclasses, activation='softmax')(model)
    model = Model(inputs=feature_input, outputs=output)
    print(model.summary())
    return(model)

In [43]:
model = build_dnn((5,), len(one_hot_labels[0]))

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dense_11 (Dense)             (None, 64)                384       
_________________________________________________________________
dense_12 (Dense)             (None, 16)                1040      
_________________________________________________________________
dense_13 (Dense)             (None, 4)                 68        
_________________________________________________________________
dense_14 (Dense)             (None, 59)                295       
Total params: 1,787
Trainable params: 1,787
Non-trainable params: 0
_________________________________________________________________
None


In [44]:
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0001), metrics=['accuracy'])

model.fit(X_train, y_train, validation_split=0.2, epochs=8)
model.evaluate(X_test, y_test, verbose=0)


Epoch 1/8
33/33 [==============================] - 0s 5ms/step - loss: 0.2832 - accuracy: 0.0201 - val_loss: 0.2837 - val_accuracy: 0.0115
Epoch 2/8
33/33 [==============================] - 0s 3ms/step - loss: 0.2707 - accuracy: 0.0249 - val_loss: 0.2820 - val_accuracy: 0.0115
Epoch 3/8
33/33 [==============================] - 0s 2ms/step - loss: 0.2703 - accuracy: 0.0412 - val_loss: 0.2820 - val_accuracy: 0.0115
Epoch 4/8
33/33 [==============================] - 0s 4ms/step - loss: 0.2703 - accuracy: 0.0421 - val_loss: 0.2819 - val_accuracy: 0.0115
Epoch 5/8
33/33 [==============================] - 0s 3ms/step - loss: 0.2702 - accuracy: 0.0421 - val_loss: 0.2820 - val_accuracy: 0.0115
Epoch 6/8
33/33 [==============================] - 0s 3ms/step - loss: 0.2702 - accuracy: 0.0460 - val_loss: 0.2819 - val_accuracy: 0.0115
Epoch 7/8
33/33 [==============================] - 0s 3ms/step - loss: 0.2701 - accuracy: 0.0441 - val_loss: 0.2819 - val_accuracy: 0.0115
Epoch 8/8
33/33 [==========

[0.27423790097236633, 0.030581040307879448]

In [45]:
predic = model.predict(X_test)
print(predic)
predic = np.argmax(predic, axis=1)
print(predic)
#labels = np.argmax(y_test, axis=1)
#confusion_matrix(labels, predic)

[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 3.2098642e-25
  6.2255321e-24 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 3.5419717e-25
  7.6704685e-24 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 4.0680169e-25
  9.7164723e-24 0.0000000e+00]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 3.1045870e-25
  5.9184372e-24 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.7919269e-25
  3.1510674e-24 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 3.7622601e-25
  7.2705634e-24 0.0000000e+00]]
[13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 1